# Backpropagation

The neural network looks like this:

![title](img/NN_structure.PNG)

## Forwardpass

To begin, lets see what the neural network currently predicts given the weights and biases above and inputs of 0.05 and 0.10. To do this we’ll feed those inputs forward though the network.

\begin{equation*}
net_{h1} = w_1 * i_1 + w_2 * i_2 + b_1 * 1\\\
net_{h2} = w_3 * i_1 + w_4 * i_2 + b_1 * 1
\end{equation*}

In [1]:
x1 = 0.05
x2 = 0.10

b1 = 0.35

w11_2 = 0.15
w12_2 = 0.20
w21_2 = 0.25
w22_2 = 0.30

y1 = 0.01
y2 = 0.99

b2 = 0.60

w11_3 = 0.4
w12_3 = 0.45
w21_3 = 0.5
w22_3 = 0.55

In [2]:
z1_2 = w11_2 * x1 + w12_2 * x2 + b1 * 1
z2_2 = w21_2 * x1 + w22_2 * x2 + b1 * 1

In [3]:
z1_2, z2_2

(0.3775, 0.39249999999999996)

---
We then squash it using the logistic function to get the output of h_1:

\begin{equation*}
out_{h1} = \frac{1}{1 + e^{-net_{h1}}} \\\
\\\
out_{h1} = \frac{1}{1 + e^{-0.3775}} \\\
\end{equation*}

We do the same for h_2:

---

In [4]:
from math import e

def logistic_function(z):
    
    return 1 / (1 + e ** (-z))

In [5]:
a1_2 = logistic_function(z1_2)
a2_2 = logistic_function(z2_2)

In [6]:
a1_2, a2_2

(0.5932699921071872, 0.596884378259767)

---
We repeat this process for the output layer neurons, using the output from the hidden layer neurons as inputs.

---

\begin{equation*}
net_{o1} = w_5 * out_{h1} + w_6 * out_{h2} + b_2 * 1\\\
net_{o2} = w_7 * out_{h1} + w_8 * out_{h2} + b_2 * 1
\end{equation*}

---

In [7]:
z1_3 = w11_3 * a1_2 + w12_3 * a2_2 + b2 * 1
z2_3 = w21_3 * a1_2 + w22_3 * a2_2 + b2 * 1

In [8]:
z1_3, z2_3

(1.10590596705977, 1.2249214040964653)

---
We then squash it using the logistic function to get the output of o_1:

\begin{equation*}
out_{o1} = \frac{1}{1 + e^{-net_{o1}}} \\\
\\\
out_{h1} = \frac{1}{1 + e^{-1.10590}} \\\
\end{equation*}

We do the same for o_2:

---

In [9]:
a1_3 = logistic_function(z1_3)
a2_3 = logistic_function(z2_3)

In [10]:
a1_3, a2_3

(0.7513650695523157, 0.7729284653214625)

---
Forwardpropagation finishes here.

---

## Calculating the Total Error

---
We can now calculate the error for each output neuron using the squared error function and sum them to get the total error:

---

\begin{equation*}
\sum_{}^{} {\frac{1}{2}(target_{o1} - out_{o1})^2}
\end{equation*}

---
For example, the target output for o_1 is 0.01 but the neural network output 0.75136507, therefore its error is:

\begin{equation*}
E_{o1} = \sum_{}^{} {\frac{1}{2}(0.01 - 0.75136507)^2} = 0.274811083
\end{equation*}

Repeating this process for o_2 (remembering that the target is 0.99) we get:

\begin{equation*}
E_{o2} = 0.023560026
\end{equation*}

The total error for the neural network is the sum of these errors:

\begin{equation*}
E_{total} = E_{o1} + E_{o2}
\end{equation*}


In [11]:
E_o1 = 0.5 * (y1 - a1_3) ** 2
E_o2 = 0.5 * (y2 - a2_3) ** 2

In [12]:
E_o1, E_o2

(0.274811083176155, 0.023560025583847746)

In [13]:
E_total = E_o1 + E_o2

In [14]:
E_total

0.2983711087600027

## The Backward Pass

Our goal with backpropagation is to update each of the weights in the network so that they cause the actual output to be closer the target output, thereby minimizing the error for each output neuron and the network as a whole.

### Output Layer

Consider $w_5$. We want to know how much a change in $w_5$ affects the total error, aka $\frac{\partial E_{total}}{\partial w_{5}}$.

By applying the chain rule we know that:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{5}} = \frac{\partial E_{total}}{\partial out_{o1}} * \frac{\partial out_{o1}}{\partial net_{o1}} * \frac{\partial net_{o1}}{\partial w_{5}}
\end{equation*}

Visually, here’s what we’re doing:

![title](img/backwardpass.PNG)

We need to figure out each piece in this equation.

<ol>
<li>
<b>First, how much does the total error change with respect to the output?</b>

\begin{equation*}
E_{total} = \frac{1}{2}(target_{o1} - out_{o1})^{2} + \frac{1}{2}(target_{o2} - out_{o2})^{2} \\\
\\\
\frac{\partial E_{total}}{\partial out_{o1}} = 2 * \frac{1}{2}(target_{o1} - out_{o1})^{2 - 1} * -1 + 0 \\\
\\\
\frac{\partial E_{total}}{\partial out_{o1}} = -(target_{o1} - out_{o1}) = -(0.01 - 0.75136507) = 0.74136507 \\\
\end{equation*}

<br>

---

</li>
<li>
<b>Next, how much does the output of $o_1$ change with respect to its total net input?</b>

The partial derivative of the logistic function is the output multiplied by 1 minus the output:

\begin{equation*}
out_{o1} = \frac{1}{1+e^{-net_{o1}}} \\\
\\\
\frac{\partial out_{o1}}{\partial net_{o1}} = out_{o1}(1 - out_{o1}) = 0.75136507(1 - 0.75136507) = 0.186815602 \\\
\end{equation*}

<br>

---

</li>
<li>
<b>Finally, how much does the total net input of $net_{o1}$ change with respect to $w_5$?</b>

\begin{equation*}
net_{o1} = w_5 * out_{h1} + w_6 * out_{h2} + b_2 * 1 \\\
\\\
\frac{\partial net_{o1}}{\partial w_{5}} = 1 * out_{h1} * w_5^{(1 - 1)} + 0 + 0 = out_{h1} = 0.593269992 \\\
\end{equation*}

<br>

---

</li>
</ol>
<b>Putting it all together:</b>

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{5}} = \frac{\partial E_{total}}{\partial out_{o1}} * \frac{\partial out_{o1}}{\partial net_{o1}} * \frac{\partial net_{o1}}{\partial w_{5}}\\\
\\\
\frac{\partial E_{total}}{\partial w_{5}} = 0.74136507 * 0.186815602 * 0.593269992 = 0.082167041
\\\
\end{equation*}

<br>
You’ll often see this calculation combined in the form of the [delta rule](https://en.wikipedia.org/wiki/Delta_rule):

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{5}} = -(target_{o1} - out_{o1}) * out_{o1}(1 - out_{o1}) * out_{h1}
\end{equation*}

<br>
Alternatively, we have $\frac{\partial E_{total}}{\partial out_{o1}}$ and $ \frac{\partial E_{total}}{\partial net_{o1}}$ which can be written as $\frac{\partial E_{total}}{\partial net_{o1}}$, aka $ \delta_{o1}$ (the Greek letter delta) aka the node delta. We can use this to rewrite the calculation above:

\begin{equation*}
\delta_{o1} = \frac{\partial E_{total}}{\partial out_{o1}} * \frac{\partial out_{o1}}{\partial net_{o1}} = \frac{\partial E_{total}}{\partial net_{o1}}
\end{equation*}

\begin{equation*}
\delta_{o1} = -(target_{o1} - out_{o1}) * out_{o1}(1 - out_{o1})
\end{equation*}

<br>
Therefore:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{5}} = \delta_{o1} out_{h1}
\end{equation*}

<br>
Some sources extract the negative sign from \delta so it would be written as:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{5}} = -\delta_{o1} out_{h1}
\end{equation*}

<br>
To decrease the error, we then subtract this value from the current weight (optionally multiplied by some learning rate, eta, which we’ll set to 0.5):

\begin{equation*}
w_5^{+} = w_5 - \eta * \frac{\partial E_{total}}{\partial w_{5}} = 0.4 - 0.5 * 0.082167041 = 0.35891648
\end{equation*}

<br>
We can repeat this process to get the new weights $w_6$, $ w_7$, and $w_8$:

$w_6^{+} = 0.408666186$<br>
$w_7^{+} = 0.511301270$<br>
$w_8^{+} = 0.561370121$<br>

We perform the actual updates in the neural network after we have the new weights leading into the hidden layer neurons (ie, we use the original weights, not the updated weights, when we continue the backpropagation algorithm below).

In [15]:
delta_y1 = -(y1 - a1_3) * a1_3 * (1 - a1_3)
delta_y2 = -(y2 - a2_3) * a2_3 * (1 - a2_3)

In [16]:
-(y1 - a1_3), -(y2 - a2_3) 

(0.7413650695523157, -0.21707153467853746)

In [17]:
a1_3 * (1 - a1_3), a2_3 * (1 - a2_3)

(0.18681560180895948, 0.17551005281727122)

In [18]:
delta_y1, delta_y2

(0.13849856162855695, -0.038098236516556236)

In [19]:
w11_3_contrib = delta_y1 * a1_2
w12_3_contrib = delta_y1 * a2_2

w21_3_contrib = delta_y2 * a1_2
w22_3_contrib = delta_y2 * a2_2

b2_contrib = delta_y1 + delta_y2

In [20]:
w11_3_contrib, w12_3_contrib, w21_3_contrib, w22_3_contrib, b2_contrib

(0.08216704056423077,
 0.08266762784753325,
 -0.02260254047747507,
 -0.022740242215978222,
 0.10040032511200071)

In [21]:
alfa = 0.5

w11_3_updated = w11_3 - alfa * w11_3_contrib
w12_3_updated = w12_3 - alfa * w12_3_contrib
w21_3_updated = w21_3 - alfa * w21_3_contrib
w22_3_updated = w22_3 - alfa * w22_3_contrib

b2_updated = b2 - alfa * b2_contrib

In [22]:
w11_3_updated, w12_3_updated, w21_3_updated, w22_3_updated, b2_updated

(0.35891647971788465,
 0.4086661860762334,
 0.5113012702387375,
 0.5613701211079891,
 0.5497998374439996)

### Hidden Layer

Next, we’ll continue the backwards pass by calculating new values for $w_1$, $w_2$, $w_3$, and $w_4$.

Big picture, here’s what we need to figure out:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{1}} = \frac{\partial E_{total}}{\partial out_{h1}} * \frac{\partial out_{h1}}{\partial net_{h1}} * \frac{\partial net_{h1}}{\partial w_{1}}
\end{equation*}

Visually:

![title](img/hidden_layer_bp.PNG)

We’re going to use a similar process as we did for the output layer, but slightly different to account for the fact that the output of each hidden layer neuron contributes to the output (and therefore error) of multiple output neurons. We know that $out_{h1}$ affects both $out_{o1}$ and $out_{o2}$ therefore the $\frac{\partial E_{total}}{\partial out_{h1}}$ needs to take into consideration its effect on the both output neurons:

\begin{equation*}
\frac{\partial E_{total}}{\partial out_{h1}} = \frac{\partial E_{o1}}{\partial out_{h1}} + \frac{\partial E_{o2}}{\partial out_{h1}}
\end{equation*}

<br>
Starting with $\frac{\partial E_{o1}}{\partial out_{h1}}$:

\begin{equation*}
\frac{\partial E_{o1}}{\partial out_{h1}} = \frac{\partial E_{o1}}{\partial net_{o1}} * \frac{\partial net_{o1}}{\partial out_{h1}}
\end{equation*}

<br>
We can calculate $\frac{\partial E_{o1}}{\partial net_{o1}}$ using values we calculated earlier:

\begin{equation*}
\frac{\partial E_{o1}}{\partial net_{o1}} = \frac{\partial E_{o1}}{\partial out_{o1}} * \frac{\partial out_{o1}}{\partial net_{o1}} = 0.74136507 * 0.186815602 = 0.18498562
\end{equation*}

<br>
And $\frac{\partial net_{o1}}{\partial out_{h1}}$ is equal to $w_5$:

\begin{equation*}
net_{o1} = w_5 * out_{h1} + w_6 * out_{h2} + b_2 * 1 \\\
\\\
\frac{\partial net_{o1}}{\partial out_{h1}} = w_5 = 0.40 \\\
\end{equation*}

<br>
Plugging them in:

\begin{equation*}
\frac{\partial E_{o1}}{\partial out_{h1}} = \frac{\partial E_{o1}}{\partial net_{o1}} * \frac{\partial net_{o1}}{\partial out_{h1}} = 0.138498562 * 0.40 = 0.055399425
\end{equation*}

<br>
Following the same process for $\frac{\partial E_{o2}}{\partial out_{h1}}$, we get:

\begin{equation*}
\frac{\partial E_{o2}}{\partial out_{h1}} = -0.019049119
\end{equation*}

Therefore:

\begin{equation*}
\frac{\partial E_{total}}{\partial out_{h1}} = \frac{\partial E_{o1}}{\partial out_{h1}} + \frac{\partial E_{o2}}{\partial out_{h1}} = 0.055399425 + -0.019049119 = 0.036350306
\end{equation*}

<br>
Now that we have $\frac{\partial E_{total}}{\partial out_{h1}}$, we need to figure out $\frac{\partial out_{h1}}{\partial net_{h1}}$ and then $\frac{\partial net_{h1}}{\partial w}$ for each weight:

\begin{equation*}
out_{h1} = \frac{1}{1+e^{-net_{h1}}} \\\
\\\
\frac{\partial out_{h1}}{\partial net_{h1}} = out_{h1}(1 - out_{h1}) = 0.59326999(1 - 0.59326999 ) = 0.241300709 \\\
\end{equation*}

We calculate the partial derivative of the total net input to h_1 with respect to w_1 the same as we did for the output neuron:

\begin{equation*}
net_{h1} = w_1 * i_1 + w_3 * i_2 + b_1 * 1 \\\
\\\
\frac{\partial net_{h1}}{\partial w_1} = i_1 = 0.05 \\\
\end{equation*}


<br>
Putting it all together:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{1}} = \frac{\partial E_{total}}{\partial out_{h1}} * \frac{\partial out_{h1}}{\partial net_{h1}} * \frac{\partial net_{h1}}{\partial w_{1}} \\\
\\\
\frac{\partial E_{total}}{\partial w_{1}} = 0.036350306 * 0.241300709 * 0.05 = 0.000438568 \\\
\end{equation*}

<br>

***

You might also see this written as:

\begin{equation*}
\frac{\partial E_{total}}{\partial w_{1}} = (\sum\limits_{o}{\frac{\partial E_{total}}{\partial out_{o}} * \frac{\partial out_{o}}{\partial net_{o}} * \frac{\partial net_{o}}{\partial out_{h1}}}) * \frac{\partial out_{h1}}{\partial net_{h1}} * \frac{\partial net_{h1}}{\partial w_{1}} \\\
\\\
\frac{\partial E_{total}}{\partial w_{1}} = (\sum\limits_{o}{\delta_{o} * w_{ho}}) * out_{h1}(1 - out_{h1}) * i_{1} \\\
\\\
\frac{\partial E_{total}}{\partial w_{1}} = \delta_{h1}i_{1}\\\
\end{equation*}

---

We can now update $w_1$:

\begin{equation*}
w_1^{+} = w_1 - \eta * \frac{\partial E_{total}}{\partial w_{1}} = 0.15 - 0.5 * 0.000438568 = 0.149780716
\end{equation*}

<br>
Repeating this for $w_2$, $w_3$, and $w_4$:

\begin{equation*}
w_2^{+} = 0.19956143 \\\
w_3^{+} = 0.24975114 \\\
w_4^{+} = 0.29950229 \\\
\end{equation*}

<br>
Finally, we’ve updated all of our weights! When we fed forward the 0.05 and 0.1 inputs originally, the error on the network was 0.298371109. After this first round of backpropagation, the total error is now down to 0.291027924. It might not seem like much, but after repeating this process 10,000 times, for example, the error plummets to 0.0000351085. At this point, when we feed forward 0.05 and 0.1, the two outputs neurons generate 0.015912196 (vs 0.01 target) and 0.984065734 (vs 0.99 target).

In [23]:
# dEtotal/da1_3 = dE_o1/da1_3 + dE_o2/da2_3
# tackling dE_o1/da1_3 part:

dEo1_da1_3 = -1 * (y1 - a1_3)

#tackling dE_o2/dout_o2 part:

dEo2_da2_3 = -1 * (y2 - a2_3)

# adding Eo1 and Eo2 to find Etotal

Etotal = dEo1_da1_3 + dEo2_da2_3

In [24]:
# da1_3/dz1_3

da1_3_dz1_3 = a1_3 * (1 - a1_3)
da2_3_dz2_3 = a2_3 * (1 - a2_3)

In [25]:
# dz1_3/da1_2

dz1_3_da1_2 = w11_3
dz2_3_da1_2 = w21_3

dz1_3_da2_2 = w12_3
dz2_3_da2_2 = w22_3

In [26]:
# da1_2/dz1_2

da1_2_dz1_2 = a1_2 * (1 - a1_2)
da2_2_dz2_2 = a2_2 * (1 - a2_2)

In [27]:
# dz1_2/dw11_2

dz1_2_dw11_2 = x1
dz1_2_dw12_2 = x2

dz2_2_dw21_2 = x1
dz2_2_dw22_2 = x2

dz1_2_db1 = 1
dz2_2_db1 = 1

In [28]:
#putting them all together


w11_2_contrib = (
    (
        (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da1_2) + 
        (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da1_2)
    )
    * da1_2_dz1_2 * dz1_2_dw11_2
)


w12_2_contrib = (
    (
        (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da1_2) + 
        (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da1_2)
    )
    * da1_2_dz1_2 * dz1_2_dw12_2
)


w21_2_contrib = (
    (
        (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da2_2) + 
        (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da2_2)
    ) 
    * da2_2_dz2_2 * dz2_2_dw21_2
)


w22_2_contrib = (
    (
        (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da2_2) + 
        (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da2_2)
    ) 
    * da2_2_dz2_2 * dz2_2_dw22_2
)

b1_contrib = (
    (
        delta_y1 * (
            (dz1_3_da1_2 * da1_2_dz1_2 * dz1_2_db1) + 
            (dz1_3_da2_2 * da2_2_dz2_2 * dz2_2_db1)
        ) +
        delta_y2 * (
            (dz2_3_da1_2 * da1_2_dz1_2 * dz1_2_db1) +
            (dz2_3_da2_2 * da2_2_dz2_2 * dz2_2_db1)
        )
    )
)

In [29]:
w11_2_contrib, w12_2_contrib, w21_2_contrib, w22_2_contrib, b1_contrib

(0.00043856773447434685,
 0.0008771354689486937,
 0.0004977127352608601,
 0.0009954254705217202,
 0.018725609394704128)

In [30]:
w11_2_updated = w11_2 - alfa * w11_2_contrib
w12_2_updated = w12_2 - alfa * w12_2_contrib
w21_2_updated = w21_2 - alfa * w21_2_contrib
w22_2_updated = w22_2 - alfa * w22_2_contrib

b1_updated = b1 - alfa * b1_contrib

In [31]:
w11_2_updated, w12_2_updated, w21_2_updated, w22_2_updated, b1_updated

(0.1497807161327628,
 0.19956143226552567,
 0.24975114363236958,
 0.29950228726473915,
 0.3406371953026479)

In [32]:

def epoch(w11_2,
          w12_2,
          w21_2,
          w22_2,
          b1,
          w11_3,
          w12_3,
          w21_3,
          w22_3,
          b2):
    
    z1_2 = w11_2 * x1 + w12_2 * x2 + b1 * 1
    z2_2 = w21_2 * x1 + w22_2 * x2 + b1 * 1
    
    a1_2 = logistic_function(z1_2)
    a2_2 = logistic_function(z2_2)
    
    z1_3 = w11_3 * a1_2 + w12_3 * a2_2 + b2 * 1
    z2_3 = w21_3 * a1_2 + w22_3 * a2_2 + b2 * 1
    
    a1_3 = logistic_function(z1_3)
    a2_3 = logistic_function(z2_3)
    
    E_o1 = 0.5 * (y1 - a1_3) ** 2
    E_o2 = 0.5 * (y2 - a2_3) ** 2
    
    E_total = E_o1 + E_o2
    
    delta_y1 = -(y1 - a1_3) * a1_3 * (1 - a1_3)
    delta_y2 = -(y2 - a2_3) * a2_3 * (1 - a2_3)
    
    w11_3_contrib = delta_y1 * a1_2
    w12_3_contrib = delta_y1 * a2_2

    w21_3_contrib = delta_y2 * a1_2
    w22_3_contrib = delta_y2 * a2_2
    
    b2_contrib = delta_y1 + delta_y2
    
    alfa = 0.5

    w11_3_updated = w11_3 - alfa * w11_3_contrib
    w12_3_updated = w12_3 - alfa * w12_3_contrib
    w21_3_updated = w21_3 - alfa * w21_3_contrib
    w22_3_updated = w22_3 - alfa * w22_3_contrib

    b2_updated = b2 - alfa * b2_contrib
    
    # dEtotal/da1_3 = dE_o1/da1_3 + dE_o2/da2_3
    # tackling dE_o1/da1_3 part:

    dEo1_da1_3 = -1 * (y1 - a1_3)

    #tackling dE_o2/dout_o2 part:

    dEo2_da2_3 = -1 * (y2 - a2_3)

    # adding Eo1 and Eo2 to find Etotal

    Etotal = dEo1_da1_3 + dEo2_da2_3
    print(Etotal)
    
    # da1_3/dz1_3

    da1_3_dz1_3 = a1_3 * (1 - a1_3)
    da2_3_dz2_3 = a2_3 * (1 - a2_3)
    
    # dz1_3/da1_2

    dz1_3_da1_2 = w11_3
    dz2_3_da1_2 = w21_3

    dz1_3_da2_2 = w12_3
    dz2_3_da2_2 = w22_3

    # da1_2/dz1_2

    da1_2_dz1_2 = a1_2 * (1 - a1_2)
    da2_2_dz2_2 = a2_2 * (1 - a2_2)

    # dz1_2/dw11_2

    dz1_2_dw11_2 = x1
    dz1_2_dw12_2 = x2

    dz2_2_dw21_2 = x1
    dz2_2_dw22_2 = x2

    dz1_2_db1 = 1
    dz2_2_db1 = 1

    #putting them all together


    w11_2_contrib = (
        (
            (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da1_2) + 
            (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da1_2)
        )
        * da1_2_dz1_2 * dz1_2_dw11_2
    )


    w12_2_contrib = (
        (
            (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da1_2) + 
            (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da1_2)
        )
        * da1_2_dz1_2 * dz1_2_dw12_2
    )


    w21_2_contrib = (
        (
            (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da2_2) + 
            (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da2_2)
        ) 
        * da2_2_dz2_2 * dz2_2_dw21_2
    )


    w22_2_contrib = (
        (
            (dEo1_da1_3 * da1_3_dz1_3 * dz1_3_da2_2) + 
            (dEo2_da2_3 * da2_3_dz2_3 * dz2_3_da2_2)
        ) 
        * da2_2_dz2_2 * dz2_2_dw22_2
    )

    b1_contrib = (
        (
            delta_y1 * (
                (dz1_3_da1_2 * da1_2_dz1_2 * dz1_2_db1) + 
                (dz1_3_da2_2 * da2_2_dz2_2 * dz2_2_db1)
            ) +
            delta_y2 * (
                (dz2_3_da1_2 * da1_2_dz1_2 * dz1_2_db1) +
                (dz2_3_da2_2 * da2_2_dz2_2 * dz2_2_db1)
            )
        )
    )

    w11_2_updated = w11_2 - alfa * w11_2_contrib
    w12_2_updated = w12_2 - alfa * w12_2_contrib
    w21_2_updated = w21_2 - alfa * w21_2_contrib
    w22_2_updated = w22_2 - alfa * w22_2_contrib

    b1_updated = b1 - alfa * b1_contrib
    
    return (w11_2_updated, w12_2_updated, w21_2_updated, w22_2_updated, b1_updated,
            w11_3_updated, w12_3_updated, w21_3_updated, w22_3_updated, b2_updated)

In [33]:
for _ in range(20000):
    w11_2,w12_2,w21_2,w22_2,b1,w11_3,w12_3,w21_3,w22_3,b2 = epoch(w11_2,w12_2,w21_2,w22_2,b1,w11_3,w12_3,w21_3,w22_3,b2)

0.5242935348737783
0.4980088204565475
0.4705523481062296
0.442177892432973
0.4131983545075514
0.38397136400439
0.35487834733802226
0.32629975609923734
0.2985901929662945
0.2720572189418917
0.24694665181318665
0.223435567251562
0.20163258415516705
0.18158384445537784
0.16328258787580385
0.14668029726950638
0.13169782278921283
0.11823544542566095
0.10618134579674321
0.09541832375508758
0.08582885605161678
0.0772987060673146
0.06971934575526034
0.06298944706366844
0.05701567201456431
0.05171295282864469
0.0470044149821921
0.04282106132215613
0.03910130606211698
0.03579042383477821
0.03283996046277299
0.030207137937482698
0.027854275443284726
0.025748240388149435
0.023859937672683962
0.022163841339053775
0.020637569885461338
0.019261504596880613
0.018018448986454688
0.01689332667942281
0.015872914662368787
0.01494560865944558
0.014101217405293354
0.013330782703507238
0.012626422347353977
0.011981193205594293
0.011388972019319377
0.01084435170106654
0.010342551165396036
0.009879336944615208

8.009991538599458e-05
8.008103803294232e-05
8.006216944612433e-05
8.004330961886887e-05
8.002445854517724e-05
8.000561621791628e-05
7.998678263105609e-05
7.996795777783124e-05
7.994914165182672e-05
7.9930334246801e-05
7.991153555603724e-05
7.98927455732696e-05
7.987396429190961e-05
7.985519170543819e-05
7.983642780778379e-05
7.981767259210466e-05
7.979892605208296e-05
7.978018818161244e-05
7.976145897394503e-05
7.974273842278716e-05
7.97240265215781e-05
7.970532326419429e-05
7.968662864412356e-05
7.966794265501335e-05
7.96492652904348e-05
7.963059654421227e-05
7.961193640991689e-05
7.959328488100527e-05
7.95746419511665e-05
7.955600761424923e-05
7.953738186376562e-05
7.951876469359208e-05
7.950015609717483e-05
7.948155606829661e-05
7.946296460080264e-05
7.944438168812872e-05
7.942580732408881e-05
7.940724150237548e-05
7.938868421679228e-05
7.937013546106646e-05
7.935159522872055e-05
7.93330635137559e-05
7.931454030968808e-05
7.929602561063294e-05
7.927751940970361e-05
7.925902170130734

4.0278803522293335e-05
4.027332346519599e-05
4.026784460499193e-05
4.0262366941681144e-05
4.0256890474803936e-05
4.025141520359529e-05
4.024594112796501e-05
4.024046824765461e-05
4.023499656170132e-05
4.022952606992994e-05
4.022405677227281e-05
4.021858866778624e-05
4.021312175629156e-05
4.0207656037358555e-05
4.020219151071487e-05
4.019672817573948e-05
4.019126603185298e-05
4.018580507912303e-05
4.018034531670135e-05
4.0174886744468244e-05
4.016942936165696e-05
4.016397316810791e-05
4.015851816355914e-05
4.015306434707912e-05
4.0147611718745896e-05
4.014216027811886e-05
4.013671002434799e-05
4.013126095754084e-05
4.0125813076753725e-05
4.012036638205256e-05
4.01149208728354e-05
4.010947654855927e-05
4.010403340915479e-05
4.0098591453829185e-05
4.0093150682096734e-05
4.008771109404764e-05
4.008227268894292e-05
4.0076835466413066e-05
4.007139942611461e-05
4.006596456735019e-05
4.006053089005909e-05
4.005509839370182e-05
4.004966707795744e-05
4.004423694225871e-05
4.003880798621705e-05
4

2.3678141479202552e-05
2.3675611326530108e-05
2.3673081529656387e-05
2.367055208811128e-05
2.3668023002229588e-05
2.3665494271782328e-05
2.3662965896561333e-05
2.36604378766863e-05
2.3657910212035796e-05
2.3655382902301042e-05
2.3652855947598264e-05
2.3650329348024607e-05
2.364780310318282e-05
2.3645277213305352e-05
2.3642751677840565e-05
2.3640226497262037e-05
2.363770167125058e-05
2.3635177199842622e-05
2.3632653082607952e-05
2.3630129319916066e-05
2.362760591134022e-05
2.3625082857040014e-05
2.3622560156932176e-05
2.362003781081548e-05
2.3617515818743703e-05
2.361499418052429e-05
2.3612472895997647e-05
2.3609951965413573e-05
2.3607431388584718e-05
2.3604911165230058e-05
2.360239129520214e-05
2.359987177885485e-05
2.3597352615813486e-05
2.3594833806249785e-05
2.3592315349750884e-05
2.358979724641913e-05
2.3587279496041155e-05
2.3584762098844206e-05
2.3582245054406745e-05
2.3579728363053165e-05
2.3577212024275193e-05
2.3574696038471815e-05
2.3572180405070572e-05
2.3569665124258815e-05

1.6018688737576975e-05
1.6017127272201573e-05
1.601556598433175e-05
1.6014004873599746e-05
1.601244394019638e-05
1.601088318395165e-05
1.6009322605084128e-05
1.6007762203307588e-05
1.600620197880244e-05
1.6004641931561744e-05
1.6003082061177842e-05
1.6001522368195437e-05
1.5999962852014313e-05
1.599840351293111e-05
1.5996844351037764e-05
1.5995285365853762e-05
1.59937265576688e-05
1.599216792666676e-05
1.5990609472264772e-05
1.5989051195108156e-05
1.59874930945169e-05
1.5985935170631516e-05
1.5984377423812826e-05
1.5982819853543884e-05
1.5981262460157755e-05
1.5979705243508724e-05
1.597814820344587e-05
1.5976591340177357e-05
1.5975034653451653e-05
1.597347814324794e-05
1.5971921809734488e-05
1.5970365652710067e-05
1.5968809672242332e-05
1.5967253868083217e-05
1.596569824053977e-05
1.596414278949229e-05
1.5962587514697923e-05
1.5961032416573e-05
1.5959477494519037e-05
1.595792274888992e-05
1.5956368179635347e-05
1.595481378641357e-05
1.595325956956807e-05
1.5951705529138743e-05
1.595015

In [34]:
def predict(w11_2,w12_2,w21_2,w22_2,b1,w11_3,w12_3,w21_3,w22_3,b2):

    z1_2 = w11_2 * x1 + w12_2 * x2 + b1 * 1
    z2_2 = w21_2 * x1 + w22_2 * x2 + b1 * 1

    a1_2 = logistic_function(z1_2)
    a2_2 = logistic_function(z2_2)

    z1_3 = w11_3 * a1_2 + w12_3 * a2_2 + b2 * 1
    z2_3 = w21_3 * a1_2 + w22_3 * a2_2 + b2 * 1

    a1_3 = logistic_function(z1_3)
    a2_3 = logistic_function(z2_3)
    
    return a1_3, a2_3

In [35]:
a1_3, a2_3 = predict(w11_2,w12_2,w21_2,w22_2,b1,w11_3,w12_3,w21_3,w22_3,b2)
a1_3, a2_3

(0.010633774372091731, 0.9893773083483602)

In [ ]:
 0.010633705710505656, 0.989377375975267